In [6]:
import mlflow
from mlflow.models import infer_signature
import mlflow.sklearn
import spacy
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score
from spacy.lang.fr.stop_words import STOP_WORDS as fr_stop

In [12]:
def build_model(
    training_set,
    pipeline,
    mlflow_run_tags=None,
    mlflow_run_parameters=None,
    mlflow_run_description=None,
    validation_set=None
):
    """
    Build a sentiment analysis model, print the evaluation result and store everything to MLFlow.

    @param: training_set: pandas dataframe containing the input training set
    @param: pipeline: scikit-learn pipeline that will be applied to the input data
    @param: mlflow_run_tags: dict of tags that will be stored in the MLFlow run
    @param: mlflow_run_parameters: dict of parameters that will be stored in the MLFlow run
    @param: mlflow_run_description: textual description of the run
    @param: validation_set: if provided, used to evaluate the model and log result in MLFlow
    @return: the trained pipeline
    """
    # Séparation des features et des labels
    X_train,y_train = training_set[0], training_set[1]
    if validation_set is not None:
        X_val,y_val  = validation_set[0],validation_set[1]
    with mlflow.start_run() as run:
        mlflow.sklearn.autolog(log_datasets=False)
        # Ajout de tags
        if mlflow_run_tags:
            for tag, value in mlflow_run_tags.items():
                mlflow.set_tag(tag, value)        
        if mlflow_run_parameters:
            for param, value in mlflow_run_parameters.items():
                mlflow.log_param(param, value)
        if mlflow_run_description:
            mlflow.set_tag("Description", mlflow_run_description)

        # Entraînement le modèle
        pipeline.fit(X_train, y_train)
        # Évaluer le modèle si le jeu de validation est fourni
        if validation_set is not None:
            y_pred = pipeline.predict(X_val)
            accuracy = accuracy_score(y_val, y_pred)
            precision = precision_score(y_val, y_pred)
            recall = recall_score(y_val, y_pred)
            
            # Logger les métriques dans MLflow
            mlflow.log_metric("accuracy_val", accuracy)
            mlflow.log_metric("precision_val", precision)
            mlflow.log_metric("recall_val", recall)
        mlflow.sklearn.log_model(pipeline, "model_pipeline")

    return pipeline


In [9]:
nlp = spacy.load("fr_core_news_sm")
train_data = pd.read_csv("../archive/train.csv")
valid_data = pd.read_csv("../archive/valid.csv")
test_data  = pd.read_csv("../archive/test.csv")

train_reviews = train_data.review.values
valid_review  = valid_data.review.values

y_val  = valid_data['polarity']
y_train = train_data['polarity']

### Experimentations


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB


mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")

mlflow.set_experiment("MLflow log_reg experience")

MAX_ITER = 500
scores = []
estimators = [('pre_processing', TfidfVectorizer(stop_words=list(fr_stop))),('lrg', LogisticRegression())]
parameters = {'C': [0.01, 0.1, 1, 10, 100], 'penalty': ['l1','l2']}
for c in parameters.get('C'):
    for pen in parameters.get('penalty'):
        lg = LogisticRegression(C=c, penalty=pen, max_iter=MAX_ITER, solver='liblinear', random_state=42)
        estimators = [('pre_processing', TfidfVectorizer(stop_words=list(fr_stop))),('lrg', lg)]
        pipe = Pipeline(estimators)
        pipe_res  = build_model((train_reviews,y_train),pipe,validation_set=(valid_review,y_val))



/Users/amadouu/M2_ML/NLP/env_nlp/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:406: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['neuf', 'qu', 'quelqu'] not in stop_words.
  warnings.warn(
2024/11/12 17:30:11 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/11/12 17:30:11 INFO mlflow.tracking._tracking_service.client: 🏃 View run secretive-wasp-751 at: http://127.0.0.1:5000/#/experiments/999691338119668531/runs/c7b822185b154413a28079b9c6fc94e1.
2024/11/12 17:30:11 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/999691338119668531.
/Users/amadouu/M2_ML/NLP/env_nlp/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:406: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizi